In [8]:
from datetime import datetime
now = datetime.now()
myTimestamp = now.strftime('%y%m%d%H%M%S')

In [9]:
Lat1 = 35.179741; Lon1 = 129.120052
Lat2 = 35.163638; Lon2 = 129.132799
slope = (Lat2-Lat1)/(Lon2-Lon1)
intercept = Lat1 - slope*Lon1  # Equation: Lat = slope * Lon + intercept
print(slope)
print(intercept)

divider = 30  # 구간을 몇분할 할지 
interval = (Lon2-Lon1)/divider

myLon = [Lon1 + x*interval for x in range(0,30,1)]
myLon = myLon + [Lon1 + x*interval for x in range(29,-1,-1)]

myLat = [slope*k + intercept for k in myLon]

# print(myLon)
# print(myLat)
print(len(myLon))
print(len(myLat))


-1.2632776339513176
198.2942147862311
60
60


In [10]:
import json

myJson_cold = {
    "data": 
    [
        {"ser_num":"PLZ2022C3","lat_c":35.169538,"lon_c":129.135419,"dof_vib":3.2,"dof_tilt":1,"err_code":0,"temp_in":8,"temp_out":24,"temp_goal":5,"cold_min_left":251,"bat_level":51,"bat_volt":371,"time_stamp":myTimestamp,"wifi_ssid":"plzhwtest","sd_mb_now":84},
    ]
}

myJson_vehicle = {
    "data": 
    [
        {"01":1,"11":"01001001001","12":"PLZ2022V3","13":"d47c44400348","16":"abcde12345","17":"354481106012345","18":"AMM574A","19":"AMM574A-10-00-LG","1A":"0612345","21":myTimestamp,"32":100,"33":1,"31":55,"34":55,"35":12,"36":1,"37":"00","38":1,"41":35.169538,"42":129.135419,"43":0.798,"44":231.8,"45":"06","46":13.07,"51":1,"52":1,"5E":1,"5F":1,"61":-75,"62":-16,"63":13,"64":-50,"65":"0034190E","66":"450","67":"08","68":"0002"},
    ]
}

print(myJson_cold)
print(myJson_vehicle)

{'data': [{'ser_num': 'PLZ2022C3', 'lat_c': 35.169538, 'lon_c': 129.135419, 'dof_vib': 3.2, 'dof_tilt': 1, 'err_code': 0, 'temp_in': 8, 'temp_out': 24, 'temp_goal': 5, 'cold_min_left': 251, 'bat_level': 51, 'bat_volt': 371, 'time_stamp': '221012222432', 'wifi_ssid': 'plzhwtest', 'sd_mb_now': 84}]}
{'data': [{'01': 1, '11': '01001001001', '12': 'PLZ2022V3', '13': 'd47c44400348', '16': 'abcde12345', '17': '354481106012345', '18': 'AMM574A', '19': 'AMM574A-10-00-LG', '1A': '0612345', '21': '221012222432', '32': 100, '33': 1, '31': 55, '34': 55, '35': 12, '36': 1, '37': '00', '38': 1, '41': 35.169538, '42': 129.135419, '43': 0.798, '44': 231.8, '45': '06', '46': 13.07, '51': 1, '52': 1, '5E': 1, '5F': 1, '61': -75, '62': -16, '63': 13, '64': -50, '65': '0034190E', '66': '450', '67': '08', '68': '0002'}]}


In [11]:
import time
import requests
from datetime import datetime

i = 0
while(True):
    now = datetime.now()
    myTimestamp = now.strftime('%y%m%d%H%M%S')

    myJson_cold['data'][0]['lat_c']=round(myLat[i], 6)
    myJson_cold['data'][0]['lon_c']=round(myLon[i], 6)
    myJson_cold['data'][0]['time_stamp']=myTimestamp
    print(myJson_cold)

    myJson_vehicle['data'][0]["41"]=round(myLat[i], 6)  #위도(latitude)
    myJson_vehicle['data'][0]["42"]=round(myLon[i], 6)  #경도(longitude)
    myJson_vehicle['data'][0]["21"]=myTimestamp  #타임스탬프
    print(myJson_vehicle)

    url_cold = "http://gps.plzlab.com/coldChain/insert"
    ser_num_cold = "PLZ2022C1"

    header_cold = {"DeviceID": ser_num_cold, "Content-Type": "application/json;charset=UTF-8"}

    x = requests.post(url_cold, json = myJson_cold, headers= header_cold)

    print(x.status_code)
    print(x.text)

    url_vehicle = "http://gps.plzlab.com/pmIot/devices/SUL01"
    ser_num_vehicle = "PLZ2022V1"

    header_vehicle = {"DeviceID": ser_num_vehicle, "Content-Type": "application/json;charset=UTF-8"}

    x = requests.post(url_vehicle, json = myJson_vehicle, headers= header_vehicle)

    print(x.status_code)
    print(x.text)
    
    i = i+1
    if(i>=60):
        i=0
        break
    time.sleep(5)

{'data': [{'ser_num': 'PLZ2022C3', 'lat_c': 35.179741, 'lon_c': 129.120052, 'dof_vib': 3.2, 'dof_tilt': 1, 'err_code': 0, 'temp_in': 8, 'temp_out': 24, 'temp_goal': 5, 'cold_min_left': 251, 'bat_level': 51, 'bat_volt': 371, 'time_stamp': '221012222436', 'wifi_ssid': 'plzhwtest', 'sd_mb_now': 84}]}
{'data': [{'01': 1, '11': '01001001001', '12': 'PLZ2022V3', '13': 'd47c44400348', '16': 'abcde12345', '17': '354481106012345', '18': 'AMM574A', '19': 'AMM574A-10-00-LG', '1A': '0612345', '21': '221012222436', '32': 100, '33': 1, '31': 55, '34': 55, '35': 12, '36': 1, '37': '00', '38': 1, '41': 35.179741, '42': 129.120052, '43': 0.798, '44': 231.8, '45': '06', '46': 13.07, '51': 1, '52': 1, '5E': 1, '5F': 1, '61': -75, '62': -16, '63': 13, '64': -50, '65': '0034190E', '66': '450', '67': '08', '68': '0002'}]}
200
{"data":[{"ser_num":"PLZ2022C3","lat_c":35.179741,"lon_c":129.120052,"dof_vib":3.2,"dof_tilt":1,"err_code":0,"temp_in":8,"temp_out":24,"temp_goal":5,"cold_min_left":251,"bat_level":51,

{'data': [{'ser_num': 'PLZ2022C3', 'lat_c': 35.175447, 'lon_c': 129.123451, 'dof_vib': 3.2, 'dof_tilt': 1, 'err_code': 0, 'temp_in': 8, 'temp_out': 24, 'temp_goal': 5, 'cold_min_left': 251, 'bat_level': 51, 'bat_volt': 371, 'time_stamp': '221012222517', 'wifi_ssid': 'plzhwtest', 'sd_mb_now': 84}]}
{'data': [{'01': 1, '11': '01001001001', '12': 'PLZ2022V3', '13': 'd47c44400348', '16': 'abcde12345', '17': '354481106012345', '18': 'AMM574A', '19': 'AMM574A-10-00-LG', '1A': '0612345', '21': '221012222517', '32': 100, '33': 1, '31': 55, '34': 55, '35': 12, '36': 1, '37': '00', '38': 1, '41': 35.175447, '42': 129.123451, '43': 0.798, '44': 231.8, '45': '06', '46': 13.07, '51': 1, '52': 1, '5E': 1, '5F': 1, '61': -75, '62': -16, '63': 13, '64': -50, '65': '0034190E', '66': '450', '67': '08', '68': '0002'}]}
200
{"data":[{"ser_num":"PLZ2022C3","lat_c":35.175447,"lon_c":129.123451,"dof_vib":3.2,"dof_tilt":1,"err_code":0,"temp_in":8,"temp_out":24,"temp_goal":5,"cold_min_left":251,"bat_level":51,

KeyboardInterrupt: 